# Números - Aula 5.

# Verbos irregulares - Aula 6.

In [3]:
# @title
# Cargar el Excel dado y mostrar una hoja editable con opciones de orden (columnas/filas: original o aleatorio)
import sys, subprocess
if 'google.colab' in sys.modules:
    # Este código solo se ejecutará en Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
    print("Estamos en Google Colab, habilitando el widget...")
    # para desactivar: output.disable_custom_widget_manager()


try:
    import ipysheet
    import ipywidgets as widgets
    import pandas as pd
    import numpy as np
    import unicodedata
    import random
    from IPython.display import display, Markdown, clear_output, HTML
except Exception:
    print('Instalando dependencias necesarias (ipysheet, ipywidgets, openpyxl) — espera...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipysheet', 'ipywidgets', 'openpyxl'])
    import ipysheet
    import ipywidgets as widgets
    import pandas as pd
    import numpy as np
    import unicodedata
    import random
    from IPython.display import display, Markdown, clear_output, HTML

# Ruta del archivo Excel (usa el que subiste)
excel_path = 'https://docs.google.com/spreadsheets/d/1-74j-PQVdV2_r8k0Tjs4NKUmCpKrpF79/export?format=xlsx'
# Leer archivo (df contiene las respuestas correctas)
df = pd.read_excel(excel_path)

# Configurar variables globales que usarán la hoja y el orden actual
rows = df.shape[0]
cols = df.shape[1]
col_headers = [str(c) for c in df.columns]
sheet = None
row_order = list(range(rows))
col_order = list(range(cols))

# Helper: normalizar texto (quitar acentos, espacios extras, y pasar a minúsculas)
def normalize_text(x):
    try:
        if x is None:
            return ''
        s = str(x).strip().lower()
        s = unicodedata.normalize('NFKD', s)
        s = ''.join(ch for ch in s if not unicodedata.combining(ch))
        return s
    except Exception:
        return str(x).strip().lower()

# Controles para elegir orden de columnas y filas (se crean antes para mostrarlos arriba)
order_cols_dropdown = widgets.Dropdown(options=[('original','original'), ('aleatorio','aleatorio')], value='original', description='orden_columnas:')
order_rows_dropdown = widgets.Dropdown(options=[('original','original'), ('aleatorio','aleatorio')], value='original', description='orden_filas:')
gen_button = widgets.Button(description='Generar hoja', button_style='primary')

# Botón para enviar y corregir (se mostrará bajo la hoja cada vez que se genere)
btn_correct = widgets.Button(description='Enviar y corregir', button_style='success')

# Función para (re)construir la hoja según opciones de orden
def build_sheet(order_cols='original', order_rows='original'):
    global sheet, row_order, col_order
    # determinar orden de filas
    if order_rows == 'aleatorio':
        row_order = random.sample(list(range(rows)), rows)
    else:
        row_order = list(range(rows))
    # determinar orden de columnas (mantener la primera columna fija en posición 0)
    if order_cols == 'aleatorio' and cols > 1:
        rest = list(range(1, cols))
        random.shuffle(rest)
        col_order = [0] + rest
    else:
        col_order = list(range(cols))
    # construir encabezados en el orden mostrado
    col_headers_order = [col_headers[k] for k in col_order]
    # limpiar salida anterior de la hoja (si existe)
    clear_output(wait=True)
    # Mostrar controles arriba
    display(widgets.HBox([order_cols_dropdown, order_rows_dropdown, gen_button]))
    display(Markdown('**Completa los espacios en blanco (como si fuera un excel, puedes usar las flechitas. Usando las flechas al parecer se regresa al inicio; esto es un bug pero si usas las flechas y luego presionas cualquier tecla, vuelves a la posición deseada), y luego haz clic en "Enviar y corregir**'))
    # crear nueva hoja
    sheet = ipysheet.sheet(rows=rows, columns=cols, column_headers=col_headers_order)
    # llenar celdas: primera columna (según orden de filas) con valor original y read-only; resto vacías
    for disp_i in range(rows):
        orig_i = row_order[disp_i]
        for disp_j in range(cols):
            orig_j = col_order[disp_j]
            if orig_j == 0:
                val = df.iat[orig_i, orig_j]
                ipysheet.cell(row=disp_i, column=disp_j, value=val, read_only=True)
            else:
                ipysheet.cell(row=disp_i, column=disp_j, value='')
    # Mostrar la hoja
    display(sheet)
    # Mostrar el botón de corrección bajo la hoja (re-attach handler if needed)
    display(btn_correct)

# Handler del botón Generar hoja
def on_gen_clicked(b):
    build_sheet(order_cols_dropdown.value, order_rows_dropdown.value)
gen_button.on_click(on_gen_clicked)

# Handler del botón Enviar y corregir: lo definimos aquí para usar row_order/col_order actuales
def submit_and_correct(b):
    display(Markdown('### Corrigiendo respuestas...'))
    try:
        completed = ipysheet.to_dataframe(sheet)
    except Exception as e:
        display(Markdown(f'Error al leer la hoja: {e}'))
        return
    result_rows = []
    total = 0
    correct = 0
    for disp_i in range(rows):
        row_res = []
        orig_i = row_order[disp_i]
        for disp_j in range(cols):
            orig_j = col_order[disp_j]
            if orig_j == 0:
                row_res.append(str(df.iat[orig_i, orig_j]))
                continue
            expected = df.iat[orig_i, orig_j]
            # leer valor ingresado en la posición mostrada
            try:
                raw = completed.iat[disp_i, disp_j]
            except Exception:
                raw = None
            # Normalizar y comparar
            matched = False
            if pd.isna(expected):
                row_display = '' if (raw is None or (isinstance(raw,float) and np.isnan(raw))) else str(raw)
                row_res.append(row_display)
                continue
            # comparar numéricos
            if (isinstance(expected, (int, np.integer)) or (isinstance(expected, float) and float(expected).is_integer())):
                try:
                    val = int(float(raw)) if raw not in (None, '') and not (isinstance(raw,float) and np.isnan(raw)) else None
                except Exception:
                    val = None
                if val is not None and val == int(expected):
                    matched = True
            elif isinstance(expected, float):
                try:
                    valf = float(raw)
                    matched = abs(valf - float(expected)) < 1e-9
                except Exception:
                    matched = False
            else:
                s_expected = normalize_text(expected)
                s_raw = normalize_text(raw)
                matched = (s_raw == s_expected and s_raw != '')
            if matched:
                correct += 1
                total += 1
                display_val = '' if (raw is None or (isinstance(raw,float) and np.isnan(raw))) else raw
                row_res.append(f'{display_val} ✅')
            else:
                total += 1
                display_val = '' if (raw is None or (isinstance(raw,float) and np.isnan(raw))) else raw
                row_res.append(f'{display_val} ❌ (esperado {expected})')
        result_rows.append(row_res)
    # Mostrar resultados con encabezados en el orden mostrado
    col_headers_order = [col_headers[k] for k in col_order]
    res_df = pd.DataFrame(result_rows, columns=col_headers_order)
    display(Markdown('### Resultados'))
    try:
        html = res_df.to_html(escape=False, index=False)
        display(HTML(f"<div style='width:100%;overflow:auto;border:1px solid #ddd;padding:6px'>{html}</div>"))
    except Exception:
        display(res_df)
    display(Markdown(f'**Aciertos:** {correct}/{total}'))

# Attach handler al botón de corrección y mostrar controles + hoja inicialmente (orden original)
btn_correct.on_click(submit_and_correct)
build_sheet('original','original')


**Completa los espacios en blanco (como si fuera un excel, puedes usar las flechitas. Usando las flechas al parecer se regresa al inicio; esto es un bug pero si usas las flechas y luego presionas cualquier tecla, vuelves a la posición deseada), y luego haz clic en "Enviar y corregir**

Sheet(cells=(Cell(column_end=0, column_start=0, read_only=True, row_end=0, row_start=0, type='text', value='Eu…

Button(button_style='success', description='Enviar y corregir', style=ButtonStyle())

To do:

- La tabla se genera antes de darle generar hoja
- También se debería mostrar únicamente las celdas en dónde te equivocaste
- Opción de reinicio para no volver a tener que correr todo el código
- Que la primera columna siempre se vea a pesar de que scrollemos horizontalmente
